# First steps with Docker and Kubernetes

Before you start learning about Kubernetes concepts in detail, let’s see how to create
a simple application, package it into a container image, and run it in a managed
Kubernetes cluster (in Google Kubernetes Engine) or in a local single-node cluster.
This should give you a slightly better overview of the whole Kubernetes system and
will make it easier to follow the next few chapters, where we’ll go over the basic
building blocks and concepts in Kubernetes.

## Creating, running, and sharing a container image

### Creating a trivial Node.js app

Now that you have a working Docker setup, you’re going to create an app. You’ll build a trivial Node.js web application and package it into a container image. The application will accept HTTP requests and respond with the hostname of the machine it’s running in. This way, you’ll see that an app running inside a container sees its own hostname and not that of the host machine, even though it’s running on the host like any other process. This will be useful later, when you deploy the app on Kubernetes and scale it out (scale it horizontally; that is, run multiple instances of the app). You’ll see your HTTP requests hitting different instances of the app.

Your app will consist of a single file called app.js with the contents shown in the following listing.

    const http = require('http');
    const os = require('os');

    console.log("Kubia server starting...");

    var handler = function(request, response) {
      console.log("Received request from " + request.connection.remoteAddress);
      response.writeHead(200);
      response.end("You've hit " + os.hostname() + "\n");
    };

    var www = http.createServer(handler);
    www.listen(8080);

It should be clear what this code does. It starts up an HTTP server on port 8080. The server responds with an HTTP response status code 200 OK and the text "You've hit <hostname>" to every request. The request handler also logs the client’s IP address to the standard output, which you’ll need later.

> The returned hostname is the server’s actual hostname, not the one the client sends in the HTTP request’s Host header.

You could now download and install Node.js and test your app directly, but this isn’t necessary, because you’ll use Docker to package the app into a container image and enable it to be run anywhere without having to download or install anything (except Docker, which does need to be installed on the machine you want to run the image on).

### Creating a Dockerfile for the image

To package your app into an image, you first need to create a file called Dockerfile, which will contain a list of instructions that Docker will perform when building the image. The Dockerfile needs to be in the same directory as the app.js file and should contain the commands shown in the following listing.

```Dockerfile
FROM node:7
ADD app.js /app.js
ENTRYPOINT ["node", "app.js"]
```

The FROM line defines the container image you’ll use as a starting point (the base image you’re building on top of). In your case, you’re using the node container image, tag 7. In the second line, you’re adding your app.js file from your local directory into the root directory in the image, under the same name (app.js). Finally, in the third line, you’re defining what command should be executed when somebody runs the image. In your case, the command is node app.js.

**Choosing a base image**

You may wonder why we chose this specific image as your base. Because your app is a Node.js app, you need your image to contain the node binary executable to run the app. You could have used any image that contains that binary, or you could have even used a Linux distro base image such as fedora or ubuntu and installed Node.js into the container at image build time. But because the node image is made specifically for running Node.js apps, and includes everything you need to run your app, you’ll use that as the base image.

### Building the container image

Now that you have your Dockerfile and the app.js file, you have everything you need to build your image. To build it, run the following Docker command:

    docker build -t kubia .

You’re telling Docker to build an image called kubia based on the contents of the current directory (note the dot at the end of the build command). Docker will look for the Dockerfile in the directory and build the image based on the instructions in the file.

**Understanding how an image is built**

The build process isn’t performed by the Docker client. Instead, the contents of the whole directory are uploaded to the Docker daemon and the image is built there. The client and daemon don’t need to be on the same machine at all. If you’re using Docker on a non-Linux OS, the client is on your host OS, but the daemon runs inside a VM. Because all the files in the build directory are uploaded to the daemon, if it contains many large files and the daemon isn’t running locally, the upload may take longer.

During the build process, Docker will first pull the base image (node:7) from the public image repository (Docker Hub), unless the image has already been pulled and is stored on your machine.

### Running the container image

    docker run --name kubia-container -p 8080:8080 -d kubia



This tells Docker to run a new container called kubia-container from the kubia image. The container will be detached from the console (-d flag), which means it will run in the background. Port 8080 on the local machine will be mapped to port 8080 inside the container (-p 8080:8080 option), so you can access the app through http://localhost:8080.

If you’re not running the Docker daemon on your local machine (if you’re using a Mac or Windows, the daemon is running inside a VM), you’ll need to use the hostname or IP of the VM running the daemon instead of localhost. You can look it up through the DOCKER_HOST environment variable.



**Accessing your app**



    curl localhost:8080

That’s the response from your app. Your tiny application is now running inside a container, isolated from everything else. As you can see, it’s returning 44d76963e8e1 as its hostname, and not the actual hostname of your host machine. The hexadecimal number is the ID of the Docker container.

### Exploring the inside of a running container

What if you want to see what the environment is like inside the container? Because multiple processes can run inside the same container, you can always run an additional process in it to see what’s inside. You can even run a shell, provided that the shell’s binary executable is available in the image.

**Running a shell inside an existing container**



The Node.js image on which you’ve based your image contains the bash shell, so you can run the shell inside the container like this:

    docker exec -it kubia-container bash

This will run bash inside the existing kubia-container container. The bash process will have the same Linux namespaces as the main container process. This allows you to explore the container from within and see how Node.js and your app see the system when running inside the container. The -it option is shorthand for two options:
- `-i`, which makes sure STDIN is kept open. You need this for entering commands into the shell.
- `-t`, which allocates a pseudo terminal (TTY).

You need both if you want the use the shell like you’re used to. (If you leave out the first one, you can’t type any commands, and if you leave out the second one, the command prompt won’t be displayed and some commands will complain about the TERM variable not being set.)

**Exploring the container from within**

Let’s see how to use the shell in the following listing to see the processes running in the container.

    ps aux

You see only three processes. You don’t see any other processes from the host OS.

**Understanding that processes in a container run in the host operating system**

If you now open another terminal and list the processes on the host OS itself, you will, among all other host processes, also see the processes running in the container, as shown in listing 2.7.

    ps aux | grep app.js

This proves that processes running in the container are running in the host OS. If you have a keen eye, you may have noticed that the processes have different IDs inside the container vs. on the host. The container is using its own PID Linux namespace and has a completely isolated process tree, with its own sequence of numbers.

### Stopping and removing a container

To stop your app, you tell Docker to stop the kubia-container container:

    docker stop kubia-container



This will stop the main process running in the container and consequently stop the container, because no other processes are running inside the container. The container itself still exists and you can see it with docker ps -a. The -a option prints out all the containers, those running and those that have been stopped. To truly remove a container, you need to remove it with the docker rm command:

    docker rm kubia-container

This deletes the container. All its contents are removed and it can’t be started again.

###  Pushing the image to an image registry

The image you’ve built has so far only been available on your local machine. To allow you to run it on any other machine, you need to push the image to an external image registry. For the sake of simplicity, you won’t set up a private image registry and will instead push the image to Docker Hub (http://hub.docker.com), which is one of the publicly available registries. Other widely used such registries are Quay.io and the Google Container Registry.

Before you do that, you need to re-tag your image according to Docker Hub’s rules. Docker Hub will allow you to push an image if the image’s repository name starts with your Docker Hub ID. You create your Docker Hub ID by registering at http://hub.docker.com. I’ll use my own ID (luksa) in the following examples. Please change every occurrence with your own ID.

**Tagging an image under an additional tag**

Once you know your ID, you’re ready to rename your image, currently tagged as kubia, to luksa/kubia (replace luksa with your own Docker Hub ID):

    docker tag kubia leon11sj/kubia

This doesn’t rename the tag; it creates an additional tag for the same image. You can confirm this by listing the images stored on your system with the docker images command, as shown in the following listing.

    docker images | head

As you can see, both kubia and luksa/kubia point to the same image ID, so they’re in fact one single image with two tags.

**Pushing the image to Docker Hub**

Before you can push the image to Docker Hub, you need to log in under your user ID with the docker login command. Once you’re logged in, you can finally push the yourid/kubia image to Docker Hub like this:

    docker push leon11sj/kubia

## Running your first app on Kubernetes

Because this may be your first time, you’ll use the simplest possible way of running an
app on Kubernetes. Usually, you’d prepare a JSON or YAML manifest, containing a
description of all the components you want to deploy, but because we haven’t talked
about the types of components you can create in Kubernetes yet, you’ll use a simple
one-line command to get something running.

### Deploying your Node.js app

The simplest way to deploy your app is to use the kubectl run command, which will
create all the necessary components without having to deal with JSON or YAML. This
way, we don’t need to dive into the structure of each object yet. Try to run the image
you created and pushed to Docker Hub earlier. Here’s how to run it in Kubernetes:

    $ kubectl run kubia --image=leon11sj/kubia --port=8080

> AWS: `kubectl run kubia --image=leon11sj/kubia --port=8080 --generator=run-pod/v1`

The --image=luksa/kubia part obviously specifies the container image you want to run, and the --port=8080 option tells Kubernetes that your app is listening on port 8080

**Introducing Pods**
 
You may be wondering if you can see your container in a list showing all the running containers. Maybe something such as kubectl get containers? Well, that’s not exactly how Kubernetes works. It doesn’t deal with individual containers directly. Instead, it uses the concept of multiple co-located containers. This group of containers is called a Pod.

A pod is a group of one or more tightly related containers that will always run together on the same worker node and in the same Linux namespace(s). Each pod is like a separate logical machine with its own IP, hostname, processes, and so on, running a single application. The application can be a single process, running in a single container, or it can be a main application process and additional supporting processes, each running in its own container. All the containers in a pod will appear to be running on the same logical machine, whereas containers in other pods, even if they’re running on the same worker node, will appear to be running on a different one.

    kubectl get pods

**Understanding what happened behind the scenes**

To help you visualize what transpired, look at figure 2.6. It shows both steps you had to perform to get a container image running inside Kubernetes. First, you built the image and pushed it to Docker Hub. This was necessary because building the image on your local machine only makes it available on your local machine, but you needed to make it accessible to the Docker daemons running on your worker nodes.

<img alt="" src="https://dpzbhybb2pdcj.cloudfront.net/luksa/Figures/02fig06_alt.jpg" data-action="zoom" data-zoom-src="https://dpzbhybb2pdcj.cloudfront.net/luksa/HighResolutionFigures/figure_2-6.png" class="medium-zoom-image">

### Accessing your web application

With your pod running, how do you access it? We mentioned that each pod gets its own IP address, but this address is internal to the cluster and isn’t accessible from outside of it. To make the pod accessible from the outside, you’ll expose it through a Service object. You’ll create a special service of type LoadBalancer, because if you create a regular service (a ClusterIP service), like the pod, it would also only be accessible from inside the cluster. By creating a LoadBalancer-type service, an external load balancer will be created and you can connect to the pod through the load balancer’s public IP.

**Creating a Service object**

    kubectl expose pod kubia --port=80 --target-port=8080 --name=kubia-service --type=LoadBalancer

**Listing services**

The expose command’s output mentions a service called kubia-http. Services are objects like Pods and Nodes, so you can see the newly created Service object by running the kubectl get services command, as shown in the following listing.

    kubectl get services

The list shows two services. Ignore the kubernetes service for now and take a close look at the kubia-http service you created. It doesn’t have an external IP address yet, because it takes time for the load balancer to be created by the cloud infrastructure Kubernetes is running on. Once the load balancer is up, the external IP address of the service should be displayed. Let’s wait a while and list the services again, as shown in the following listing.

Aha, there’s the external IP. Your application is now accessible at http://104.155.74.57:8080 from anywhere in the world.

**Accessing your service through its external IP**

You can now send requests to your pod through the service’s external IP and port:

    curl 35.246.197.19

Woohoo! Your app is now running somewhere in your three-node Kubernetes cluster (or a single-node cluster if you’re using Minikube). If you don’t count the steps required to set up the whole cluster, all it took was two simple commands to get your app running and to make it accessible to users across the world.

If you look closely, you’ll see that the app is reporting the name of the pod as its hostname. As already mentioned, each pod behaves like a separate independent machine with its own IP address and hostname. Even though the application is running in the worker node’s operating system, to the app it appears as though it’s running on a separate machine dedicated to the app itself—no other processes are running alongside it.